In [1]:
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split  

In [2]:
def cutval(df):
    topcut = []
    for d in df['簡介']:
        top = jieba.analyse.extract_tags(d,topK=2)
        topcut.append(top)

    typecut = []
    for d in df['類型']:
        ch =re.compile("[\u4e00-\u9fa5]")
        seg_word =  "".join(ch.findall(d))
        top = jieba.lcut(seg_word)
        typecut.append(top)

    namecut = []
    for d in df['中文名稱']:
        ch =re.compile("[\u4e00-\u9fa5]")
        name =  "".join(ch.findall(d))
        namecut.append(name)

    data = { 'type':typecut ,'name':namecut,'article':topcut}
    df1 = pd.DataFrame(data)

    df1['type'] = df1['type'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    df1['article'] = df1['article'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    df1['key'] = df1['name'].astype(str)+' '+df1['type'].astype(str)+' '+df1['article'].astype(str)
    return df1

In [7]:
def count(df):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['key'])
    tfidf = TfidfTransformer() 
    tf=tfidf.fit_transform(X)
    word = vectorizer.get_feature_names() 
    
    return tf

In [3]:
def knn(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y['type'].str[0:2], test_size=0.07659)
    clf=KNeighborsClassifier(n_neighbors=51)
    clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)
    y_test=y_test.values

    print("精準度：",metrics.accuracy_score(y_test, y_pred))

In [8]:
df = pd.read_csv('datamovie.csv')
df = df.drop(labels=['Unnamed: 0'],axis='columns')
df1 = cutval(df) 
tf = count(df1)
knn(tf,df1)

精準度： 0.748
